Run WIT for the MER shapefile dataset
        This notebook:
        needs to check for existence, load the shapefile, get the data, run the results, save.

In [ ]:
import datacube
import datetime
import fiona
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
import seaborn as sns
from shapely import geometry
import sys
import xarray as xr

from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from textwrap import wrap

from datacube.storage import masking
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from digitalearthau.utils import wofs_fuser

sys.path.append('/g/data/r78/rjd547/jupyter_notebooks/dea-notebooks/10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools, WetlandsTools, SpatialTools, wetlands_insight

dc = datacube.Datacube(app='wetlands insight tool')
%load_ext autoreload
%autoreload 2

In [ ]:
#set output directory
global Output_dir
Output_dir = '/g/data/r78/rjd547/CEWO_Project/MER_Veg/MER_veg_WIT/'

In [ ]:
#set polygon path
poly_path='/g/data/r78/rjd547/CEWO_Project/MER_Veg/Mer_Veg_Sites.shp'

In [ ]:
def get_MER_polyName(shapefile):
    ''' function designed specifically for the RAMSAR wetlands australia shapefile. Takes the shapefile and extracts
    the ramsar name, wetland name and objectID from the ESRI shapefile format and turns it into a useful string for our output.
    :Inputs: shapefile with RAMSAR_NAM, WETLAND_NA, and OBJECTID as properties. 
    Author: Bex Dunn Last Edited: March 2019'''
    # get the ramsar name from the shapes 
    SAMO_ID = shapefile['properties']['SAMO_ID']
    NAME = '_'.join(shapefile['properties']['NAME'].split(' '))
    polyname = f'{SAMO_ID}-{NAME}'
    print(f'processing polygon {polyname}')
    return(polyname)

In [ ]:
# with fiona.open(poly_path) as shapes:
#     #get crs object from polygon file
#     crs = geometry.CRS(shapes.crs_wkt)
#     for feature in shapes:
    
#         if feature['id'] == str(1): 
#             feature_geometry=feature['geometry']
#             polyname = get_MER_polyName(feature)            
#             geom = geometry.Geometry(feature_geometry, crs=crs)
#             query = {'geopolygon': geom}
            
#             #if os.path.exists(f'{Output_dir}{polyName}.csv')==False or os.path.exists(f'{Output_dir}{polyName}.png')==False:

## run WIT over each polygon, along with the fast version of the csv writer. 

In [ ]:
#set up colour palette for our plots
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]  

In [ ]:
with fiona.open(poly_path) as shapes:
    #get crs object from polygon file
    crs = geometry.CRS(shapes.crs_wkt)
    for feature in shapes:
        #name our polygon using the get MER polyname function    
        feature_geometry=feature['geometry']
        polyname = get_MER_polyName(feature)            
        geom = geometry.Geometry(feature_geometry, crs=crs)
        query = {'geopolygon': geom}
        
        #if this polyon has already been run, don't run it again!
        if os.path.exists(f'{Output_dir}{polyname}.csv')==False or os.path.exists(f'{Output_dir}{polyname}.png')==False:
            #get our data
            WITplots=wetlands_insight.WIT_array(query)
            #make a stacked area plot
            plt.clf()
            fig= plt.figure(figsize = (12,4))
            plt.stackplot(WITplots.water.time.values, 
                          WITplots.water, 
                          WITplots.wet, 
                          WITplots.green, 
                          WITplots.dry,
                          WITplots.bare,
                          labels=['water',
                                  'wet',
                                  'green veg',
                                  'dry veg',
                                  'bare soil',
                                 ], colors=pal, alpha = 0.6)


            #set axis limits to the min and max
            plt.axis(xmin = WITplots.water.time[0].data, xmax = WITplots.water.time[-1].data, ymin = 0, ymax = 100)

            #add a legend and a tight plot box
            plt.legend(loc='lower left', framealpha=0.6)
            #plt.tight_layout()

            #create rectangle borders for no-data times (SLC-off only)
            LS5_8_gap_start = datetime(2011,11,1)
            LS5_8_gap_end = datetime(2013,4,1)

            # convert to matplotlib date representation
            gap_start = mdates.date2num(LS5_8_gap_start)
            gap_end = mdates.date2num(LS5_8_gap_end)
            gap = gap_end - gap_start


            #set date ticks every year
            #years = mdates.YearLocator(2)
            #yearsFmt = mdates.DateFormatter('%Y')
            ax = plt.gca()
            #ax.xaxis.set_major_locator(years)
            #ax.xaxis.set_major_formatter(yearsFmt)
            #ax.yaxis.set_ticks(np.arange(0,110,10))
            ax.set_xlabel(f'The Fractional Cover algorithm developed by the Joint Remote'
            f' Sensing Research Program and \n the Water Observations from Space algorithm '
            f'developed by Geoscience Australia are used in the production of this data',style='italic')


            #set up rectangle
            slc_rectangle= Rectangle((gap_start,0), gap, 100,alpha = 0.5, facecolor=sns.xkcd_rgb['white'],
                         edgecolor=sns.xkcd_rgb['white'], hatch="////",linewidth=2)
            ax.add_patch(slc_rectangle)
            fig.autofmt_xdate()


            polyname=polyname.replace("'","\\'")
            title=ax.set_title("\n".join(wrap(f'Percentage of area WOfS, Wetness, Fractional Cover for {polyname}')))
            fig.tight_layout()
            title.set_y(1.05)

            #save the figure
            plt.savefig(f'{Output_dir}{polyname}.png')#, transparent=True)
            plt.show()
            print(f'plot created for {polyname}')

            #save a csv for the polygon
            WITcsvs= WITplots.to_dataframe().round(2) #turn the xarray into a pandas dataframe and then into a csv!
            WITcsvs.to_csv(f'{Output_dir}{polyname}.csv')
            print(f'wrote output data to file {Output_dir}{polyname}.csv')            